In [4]:
import pandas as pd

def copy_last_n_days_with_date_shift(
    csv_path: str,
    source_station: str,
    target_station: str,
    desired_last_date: str = "2025-11-03",   # can be "YYYY-MM-DD" or "YYYY-MM-DD HH:MM:SS"
    window_days: int = 30,
    out_path: str = "data_with_copied_station.csv",
    ds_col: str = "ds",
    station_col: str = "station",
    tz: str = "Asia/Bangkok",
):
    # 1. load and parse timestamps
    df = pd.read_csv(csv_path)
    df[ds_col] = pd.to_datetime(df[ds_col], errors="raise")

    # 2. ensure timestamps are tz-aware in Bangkok
    import pandas.api.types as ptypes
    if ptypes.is_datetime64tz_dtype(df[ds_col]):
        df[ds_col] = df[ds_col].dt.tz_convert(tz)
    else:
        df[ds_col] = df[ds_col].dt.tz_localize(tz)

    # 3. get source station data and its last timestamp
    src = df[df[station_col] == source_station].sort_values(by=ds_col)
    if src.empty:
        raise ValueError(f"No rows found for source station '{source_station}'")

    src_last = src[ds_col].max()

    # 4. pick the last `window_days` (rows within last window_days period up to src_last)
    cutoff = src_last - pd.Timedelta(days=window_days - 1)  # inclusive window
    src_window = src[src[ds_col] >= cutoff].copy()
    if src_window.empty:
        raise ValueError("No rows found in the requested window. "
                         "You may want to use fewer days or check the source data.")

    # 5. build desired_last
    # If user provided a time in the string (e.g. "2025-11-03 15:30:00"), use that exact datetime.
    # If user provided only a date ("2025-11-03"), preserve the time-of-day from src_last.
    user_str = str(desired_last_date)
    has_time = ":" in user_str  # simple heuristic: if there's a colon, the user included time

    parsed = pd.to_datetime(desired_last_date)  # may be tz-naive
    if has_time:
        # Use the provided date+time (then localize/convert to tz)
        desired_last = pd.Timestamp(parsed)
        # make tz-aware in Bangkok
        if desired_last.tzinfo is None:
            desired_last = desired_last.tz_localize(tz)
        else:
            desired_last = desired_last.tz_convert(tz)
    else:
        # Only a date provided -> preserve time-of-day from src_last
        desired_base = pd.Timestamp(parsed.date(), tz=tz)  # midnight at that date in tz
        desired_last = desired_base.replace(
            hour=src_last.hour,
            minute=src_last.minute,
            second=src_last.second,
            microsecond=src_last.microsecond
        )

    # 6. compute shift and apply to the selected window
    delta = desired_last - src_last
    src_window[ds_col] = src_window[ds_col] + delta

    # 7. set station to target and (optionally) change any other id fields if needed
    src_window[station_col] = target_station

    # 8. append to original df (or you can write src_window separately)
    df_out = pd.concat([df, src_window], ignore_index=True)

    # optional: sort by station then time (or any other desired sort)
    df_out.sort_values(by=[station_col, ds_col], inplace=True, ignore_index=True)

    # 9. save result
    df_out.to_csv(out_path, index=False, date_format="%Y-%m-%d %H:%M:%S%z")

    return df_out, src_window

# -------------------------
# Example usage to make the last copied timestamp = 2025-11-03 15:30:00+07:00:
df_full, new_rows = copy_last_n_days_with_date_shift(
    csv_path="/workspaces/Baswap-app/dataset/merged_all_data copy.csv",
    source_station="CanGio",
    target_station="VGU",
    desired_last_date="2025-11-04 15:30:00",
    window_days=30,
    out_path="/workspaces/Baswap-app/dataset/merged_all_data copy.csv"
)

/tmp/ipykernel_15736/240669733.py:20: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if ptypes.is_datetime64tz_dtype(df[ds_col]):


In [1]:
import pandas as pd

# adjust path/name
infile = "/workspaces/Baswap-app/dataset/merged_all_data.csv"
outfile = "/workspaces/Baswap-app/dataset/merged_all_data_new.csv"

# read CSV
df = pd.read_csv(infile)

# drop Unnamed: 0 if it's just an index column
if "Unnamed: 0" in df.columns:
    # only drop if it looks like a default index column (all integers or monotonic)
    try:
        if pd.api.types.is_integer_dtype(df["Unnamed: 0"]) or df["Unnamed: 0"].is_monotonic_increasing:
            df = df.drop(columns=["Unnamed: 0"])
    except Exception:
        # if any problem, still safe to drop if user expects it
        df = df.drop(columns=["Unnamed: 0"])

# find EC column (exact match first, else fuzzy)
ec_original = "EC[g/l]"

# coerce EC column to numeric (non-numeric -> NaN)
df[ec_original] = pd.to_numeric(df[ec_original], errors="coerce")

# keep only ds, station and EC
keep_cols = ["ds", "station", ec_original]
missing = [c for c in keep_cols if c not in df.columns]
if missing:
    raise KeyError(f"Missing required column(s): {missing}")

df = df[keep_cols].copy()

# optional: parse ds to datetime (uncomment if you want)
# df["ds"] = pd.to_datetime(df["ds"], errors="coerce")

# rename EC column
df = df.rename(columns={ec_original: "EC Value (g/l)"})

# compute EC Value (us/cm)
# multiplier is 2000 per your instruction
df["EC Value (us/cm)"] = df["EC Value (g/l)"] * 2000

# (optional) reorder columns
df = df[["ds", "station", "EC Value (g/l)", "EC Value (us/cm)"]]

# save
df.to_csv(outfile, index=False)

print(f"Saved processed file to: {outfile}")


Saved processed file to: /workspaces/Baswap-app/dataset/merged_all_data_new.csv


In [1]:
from data import combined_data_retrieve

df = combined_data_retrieve()
df.head()

2025-11-03 03:59:05.904 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-03 03:59:05.906 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 03:59:06.246 
  command:

    streamlit run /home/vscode/.local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-03 03:59:06.246 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 03:59:06.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 03:59:06.248 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 03:59:06.758 Thread 'Thread-12': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 03:59:06.760 Thread 'Thread-12': 

,ds,EC[g/l],station,Unnamed: 0,DO Value,DO Temperature,EC Value (us/cm),EC Temperature,Battery Voltage
0,1996-02-02 03:00:00,0.1,AnDinh,NaN,NaN,NaN,NaN,NaN,NaN
1,1996-02-02 05:00:00,0.1,AnDinh,NaN,NaN,NaN,NaN,NaN,NaN
2,1996-02-02 07:00:00,0.1,AnDinh,NaN,NaN,NaN,NaN,NaN,NaN
3,1996-02-04 17:00:00,0.1,AnDinh,NaN,NaN,NaN,NaN,NaN,NaN
4,1996-02-04 19:00:00,0.1,AnDinh,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df["ds"].max().date(), df["ds"].min().date()

(datetime.date(2025, 6, 6), datetime.date(1995, 2, 2))

In [3]:
# after reading df
print("dtype:", df["ds"].dtype)
print("first rows:", df["ds"].head(10).tolist())

# count actual Python types inside the Series
print(df["ds"].map(type).value_counts().to_dict())

dtype: object
first rows: ['1996-02-02 03:00:00+07:00', '1996-02-02 05:00:00+07:00', '1996-02-02 07:00:00+07:00', '1996-02-04 17:00:00+07:00', '1996-02-04 19:00:00+07:00', '1996-02-04 21:00:00+07:00', '1996-02-04 23:00:00+07:00', '1996-02-05 05:00:00+07:00', '1996-02-05 07:00:00+07:00', '1996-02-05 09:00:00+07:00']
{<class 'str'>: 863428}
